# Symmetric Matrix Vector Multiply Routines (stored in upper triangle)

This notebook walks you through how to implement $ y := A x + y $ where $ A $ is symmetric with only the upper triangular part stored.

## Getting started

We will use some functions that are part of our laff library (of which this function will become a part) as well as some routines from the FLAME API (Application Programming Interface) that allows us to write code that closely resembles how we typeset algorithms using the FLAME notation.  These functions are imported with the "import laff as laff" and "import flame" statements.

## Algorithm that takes dot products

<img src="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/4331_Symv_u_unb_var1.png" alt="Alternative matrix-vector multiplication via dot products algorithm" width="50%">

## The routine <br> <code> Symv_u_unb_var1( A, x, y ) </code>

This routine, given symmetric matrix $ A \in \mathbb{R}^{n \times n} $, $ x \in \mathbb{R}^n $, and $ y \in \mathbb{R}^n $, computes $ y := A x + y $.  The "_u_" in the name of the routine indicates that $ A $ is stored in the upper triangular part of the matrix.

The specific laff functions we will use are 
<ul>
<li> <code> laff.dots( x, y, alpha ) </code> which computes $ \alpha := x^T y + \alpha $.  </li>
</ul>

Use the <a href="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/index.html"> Spark webpage</a> to generate a code skeleton.  (Make sure you adjust the name of the routine.)

In [1]:
import flame
import laff as laff

def Symv_u_unb_var1(A, x, y):

    ATL, ATR, \
    ABL, ABR  = flame.part_2x2(A, \
                               0, 0, 'TL')

    xT, \
    xB  = flame.part_2x1(x, \
                         0, 'TOP')

    yT, \
    yB  = flame.part_2x1(y, \
                         0, 'TOP')

    while ATL.shape[0] < A.shape[0]:

        A00,  a01,     A02,  \
        a10t, alpha11, a12t, \
        A20,  a21,     A22   = flame.repart_2x2_to_3x3(ATL, ATR, \
                                                       ABL, ABR, \
                                                       1, 1, 'BR')

        x0,   \
        chi1, \
        x2    = flame.repart_2x1_to_3x1(xT, \
                                        xB, \
                                        1, 'BOTTOM')

        y0,   \
        psi1, \
        y2    = flame.repart_2x1_to_3x1(yT, \
                                        yB, \
                                        1, 'BOTTOM')

        #------------------------------------------------------------#

        laff.dots(a01, x0, psi1)
        laff.dots(alpha11, chi1, psi1)
        laff.dots(a12t, x2, psi1)

        #------------------------------------------------------------#

        ATL, ATR, \
        ABL, ABR  = flame.cont_with_3x3_to_2x2(A00,  a01,     A02,  \
                                               a10t, alpha11, a12t, \
                                               A20,  a21,     A22,  \
                                               'TL')

        xT, \
        xB  = flame.cont_with_3x1_to_2x1(x0,   \
                                         chi1, \
                                         x2,   \
                                         'TOP')

        yT, \
        yB  = flame.cont_with_3x1_to_2x1(y0,   \
                                         psi1, \
                                         y2,   \
                                         'TOP')

    flame.merge_2x1(yT, \
                    yB, y)

## Testing

Let's quickly test the routine by creating a 4 x 4 matrix and related vectors, performing the computation.

In [2]:
from numpy import random
from numpy import matrix

A = matrix( random.rand( 4,4 ) )
x = matrix( random.rand( 4,1 ) )
y = matrix( random.rand( 4,1 ) )
yold = matrix( random.rand( 4,1 ) )

print( 'A before =' )
print( A )

print( 'x before =' )
print( x )

print( 'y before =' )
print( y )

A before =
[[0.97724211 0.16358976 0.55283939 0.83493888]
 [0.89802688 0.03063654 0.19631757 0.02379361]
 [0.49986744 0.35871513 0.43625873 0.35893322]
 [0.10708223 0.45730217 0.31805344 0.99193845]]
x before =
[[0.20266324]
 [0.80909945]
 [0.56430255]
 [0.81518016]]
y before =
[[0.73042806]
 [0.63246504]
 [0.63548152]
 [0.04418958]]


In [3]:
import numpy as np

# Notice that A is not symmetric.  We now "symmetrize it"
Asymm = np.triu( A ) + np.transpose( np.triu( A, 1 ) )

laff.copy( y, yold )   # save the original vector y

Symv_u_unb_var1( A, x, y )

print( 'y after =' )
print( y )

print( 'y - ( Asymm * x + yold ) = ' )
print( y - ( Asymm * x + yold ) )

y after =
[[2.05343377]
 [0.82058526]
 [1.44513933]
 [1.24380787]]
y - ( Asymm * x + yold ) = 
[[0.00000000e+00]
 [0.00000000e+00]
 [2.22044605e-16]
 [2.22044605e-16]]


Bingo, it seems to work!  (Notice that we are doing floating point computations, which means that due to rounding you may not get an exact "0".)

## Watch your code in action!

Copy and paste the code into <a href="http://edx-org-utaustinx.s3.amazonaws.com/UT501x/PictureFlame/PictureFLAME.html"> PictureFLAME </a>, a webpage where you can watch your routine in action.  Just cut and paste into the box.  

Disclaimer: we implemented a VERY simple interpreter.  If you do something wrong, we cannot guarantee the results.  But if you do it right, you are in for a treat.

If you want to reset the problem, just click in the box into which you pasted the code and hit "next" again.

## Algorithm that uses axpys

<img src="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/4331_Symv_u_unb_var2.png" alt="Alternative matrix-vector multiplication via dot products algorithm" width="50%">

## The routine <br> <code> Symv_u_unb_var2( A, x, y ) </code>

This routine, given symmetric matrix $ A \in \mathbb{R}^{n \times n} $, $ x \in \mathbb{R}^n $, and $ y \in \mathbb{R}^n $, computes $ y := A x + y $.  The "_u_" in the name of the routine indicates that $ A $ is stored in the upper triangular part of the matrix.

The specific laff functions we will use are 
<ul>
<li> <code> laff.axpy( alpha, x, y ) </code> which computes $ y := \alpha x +  y  $.  </li>
</ul>

Use the <a href="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/index.html"> Spark webpage</a> to generate a code skeleton.  (Make sure you adjust the name of the routine.)

In [4]:
def Symv_u_unb_var2(A, x, y):

    ATL, ATR, \
    ABL, ABR  = flame.part_2x2(A, \
                               0, 0, 'TL')

    xT, \
    xB  = flame.part_2x1(x, \
                         0, 'TOP')

    yT, \
    yB  = flame.part_2x1(y, \
                         0, 'TOP')

    while ATL.shape[0] < A.shape[0]:

        A00,  a01,     A02,  \
        a10t, alpha11, a12t, \
        A20,  a21,     A22   = flame.repart_2x2_to_3x3(ATL, ATR, \
                                                       ABL, ABR, \
                                                       1, 1, 'BR')

        x0,   \
        chi1, \
        x2    = flame.repart_2x1_to_3x1(xT, \
                                        xB, \
                                        1, 'BOTTOM')

        y0,   \
        psi1, \
        y2    = flame.repart_2x1_to_3x1(yT, \
                                        yB, \
                                        1, 'BOTTOM')

        #------------------------------------------------------------#

        laff.axpy(chi1, a01, y0)
        laff.axpy(chi1, alpha11, psi1)
        laff.axpy(chi1, a12t, y2)

        #------------------------------------------------------------#

        ATL, ATR, \
        ABL, ABR  = flame.cont_with_3x3_to_2x2(A00,  a01,     A02,  \
                                               a10t, alpha11, a12t, \
                                               A20,  a21,     A22,  \
                                               'TL')

        xT, \
        xB  = flame.cont_with_3x1_to_2x1(x0,   \
                                         chi1, \
                                         x2,   \
                                         'TOP')

        yT, \
        yB  = flame.cont_with_3x1_to_2x1(y0,   \
                                         psi1, \
                                         y2,   \
                                         'TOP')

    flame.merge_2x1(yT, \
                    yB, y)

## Testing

Let's quickly test the routine by creating a 4 x 4 matrix and related vectors, performing the computation.

In [5]:
from numpy import random
from numpy import matrix

A = matrix( random.rand( 4,4 ) )
x = matrix( random.rand( 4,1 ) )
y = matrix( random.rand( 4,1 ) )
yold = matrix( random.rand( 4,1 ) )

print( 'A before =' )
print( A )

print( 'x before =' )
print( x )

print( 'y before =' )
print( y )

A before =
[[0.68545317 0.15981055 0.19070033 0.61806553]
 [0.59793712 0.28394259 0.84024905 0.23251671]
 [0.1427403  0.47633233 0.99403689 0.63172497]
 [0.80942606 0.18649068 0.92936957 0.48394524]]
x before =
[[0.71834195]
 [0.07413021]
 [0.81572648]
 [0.00457394]]
y before =
[[0.29444985]
 [0.85810004]
 [0.978895  ]
 [0.38861949]]


In [6]:
import numpy as np

# Notice that A is not symmetric.  We now "symmetrize it"
Asymm = np.triu( A ) + np.transpose( np.triu( A, 1 ) )

laff.copy( y, yold )   # save the original vector y

Symv_u_unb_var2( A, x, y )

print( 'y after =' )
print( y )

print( 'y - ( Asymm * x + yold ) = ' )
print( y - ( Asymm * x + yold ) )

y after =
[[0.95707271]
 [1.68042431]
 [1.99192258]
 [1.36736673]]
y - ( Asymm * x + yold ) = 
[[ 0.00000000e+00]
 [-2.22044605e-16]
 [ 0.00000000e+00]
 [ 0.00000000e+00]]


Bingo, it seems to work!  (Notice that we are doing floating point computations, which means that due to rounding you may not get an exact "0".)

## Watch your code in action!

Copy and paste the code into <a href="http://edx-org-utaustinx.s3.amazonaws.com/UT501x/PictureFlame/PictureFLAME.html"> PictureFLAME </a>, a webpage where you can watch your routine in action.  Just cut and paste into the box.  

Disclaimer: we implemented a VERY simple interpreter.  If you do something wrong, we cannot guarantee the results.  But if you do it right, you are in for a treat.

If you want to reset the problem, just click in the box into which you pasted the code and hit "next" again.